In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")
print(os.environ["LANGCHAIN_TRACING_V2"])

[API KEY]
sk-proj-2ALiWBzcJl4s9ri6EUJ6T3BlbkFJxbUDIanlzbIf6JMPE7o2
true


In [6]:
import torch
import os
import uuid
import json

from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts.chat import PromptTemplate, ChatPromptTemplate

from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
import transformers

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from transformers import pipeline, AutoModel, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager

In [9]:
# 단계 1 : 문서 로드
# RecursiveCharacterTextSplitter 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)

# 기존 JSON 데이터를 불러옵니다.
with open('/media/choi/HDD1/mmaction2/data/Korea_construction_standard/OLD/1000tokens_Test/LHCS_distill_id_735.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
new_data = []
    
for item in data:
    # page_content 분할
    page_content = item["page_content"]
    source = item["metadata"]["source"]
    
    # RecursiveCharacterTextSplitter로 분할
    split_texts = text_splitter.split_text(page_content)
    
    # 분할된 각 텍스트에 대해 새로운 id와 함께 저장
    for split_text in split_texts:
        
        if len(split_text.split()) >= 50:
            new_item = {
                "page_content": split_text,
                "metadata": {
                    "id": str(uuid.uuid4()),  # 새로운 고유 id 생성
                    "source": source  # 기존 source 유지
                }
            }
            new_data.append(new_item)
        
# 결과를 새로운 JSON 파일로 저장
with open('/media/choi/HDD1/mmaction2/data/Korea_construction_standard/LHCS_distill_id.json', 'w', encoding='utf-8') as f:
    json.dump(new_data, f, ensure_ascii=False, indent=4)

print(f'총 {len(new_data)}개의 512토큰 이하의 데이터 생성 완료!')

print("페이지 분할 완료 및 저장 완료")

총 1255개의 512토큰 이하의 데이터 생성 완료!
페이지 분할 완료 및 저장 완료


In [20]:
# 단계 4: LLM 생성
# EEVE 모델 사용과 문서 정제에는 gpt-4o 사용
llm = ChatOpenAI(model_name='gpt-4o', temperature=0.0)

In [21]:
# 질문 생성을 위한 prompt 생성
question_prompt = ChatPromptTemplate.from_template(
"""
- 입력된 Text 문서에서 만들 수 있는 질문을 한 가지 만들어줘.
- 입력된 문서는 한국의 건설기준에 대한 문서야.
- 질문은 입력된 문서를 가장 잘 활용한 질문 한 가지만 출력해줘.
- 질문은 문서에 있는 특별한 지식으로 구성될수록 좋으며, 숫자등 명확한 정답을 가지고 있는 질문이 좋아.
- 질문은 한글로 출력해줘.
- 여러 종류의 문서에 비슷한 내용이 있을 수 있기 때문에 문서의 종류에서 유추할 수 있는 정보도 "~공사에서," 와 같은 형태로 넣어줘

<Text>: {input}
<Text source>: {source}
----
<Question>:
"""
)

class QuestionParser(StrOutputParser):
    def parse(self, response):
        # "Question:" 이후의 텍스트를 추출
        if "Question>:" in response:
            return response.split("Question>:")[1].strip()
        return response.strip()

In [22]:
# 답변생성을 위한 prompt 생성
answer_prompt = ChatPromptTemplate.from_template(
    """
- <Question> 이후에 오는 질문은 <Text> 이후에 오는 문서와 관련된 질문이야.
- <Text> 이후의 문서를 참고해서 <Question> 이후에 오는 질문에 대한 답변을 생성해줘.
- 정확하고 간결하게 답변해줘
- 답변은 한글로 출력해줘


<Question> : {question}
<Text>: {input}
----
<Answer>:
    """
)

class AnswerParser(StrOutputParser):
    def parse(self, response):
        # "Question:" 이후의 텍스트를 추출
        if "Answer>:" in response:
            return response.split("Answer>:")[1].strip()
        return response.strip()

In [29]:
# 질문과 답변 생성하는 chain 생성 후 데이터를 불러와서 질문 답변 생성

Q_chain = question_prompt | llm | QuestionParser()
A_chain = answer_prompt | llm | AnswerParser()


with open('/media/choi/HDD1/mmaction2/data/Korea_construction_standard/LHCS_distill_id.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 새로운 데이터를 저장할 리스트
new_data = []

# 각 데이터 항목에 대해 처리
for item in data:
    # page_content 분할
    page_content = item["page_content"]
    source = item["metadata"]["source"]
    
    # 질문 생성
    question = Q_chain.invoke({"input": page_content, "source": source})
    
    # 답변 생성
    answer = A_chain.invoke({"question": question, "input": page_content})
    
    # 새로운 아이템에 question과 answer 추가
    new_item = {
        "page_content": page_content,
        "metadata": {
            "id": str(uuid.uuid4()),  # 새로운 고유 id 생성
            "source": source,  # 기존 source 유지
            "question": question,  # 생성된 질문 추가
            "answer": answer  # 생성된 답변 추가
        }
    }
    print(new_item)
    
    # break

    new_data.append(new_item)
            
# 결과를 새로운 JSON 파일로 저장
with open('/media/choi/HDD1/mmaction2/data/Korea_construction_standard/LHCS_qna_id.json', 'w', encoding='utf-8') as f:
    json.dump(new_data, f, ensure_ascii=False, indent=4)

print("페이지 분할, 질문 및 답변 생성 완료 및 저장 완료")

{'page_content': '태양광 발전 설비의 설치 \n개별 모니터링 설비 1식\n웹 모니터링 설비 1식\n태양광 발전 설비의 사용전 검사 및 설치확인 검사 수행\n태양전지판 설치 지붕층 옥상 안전난간대 설치1.4 설치 및 공사기간\n1.4.1 옥상 안전 난간대시설은 건축공사분의 옥상 안전난간대 시설과 겹칠 경우, 건축공사 분의 옥상 안전난간대를 설계변경 조치를 하고, 태양전지판이 설치되는 면만 안전난간대 시공을 한다.\n1.4.2 옥상 안전난간대를 먼저 설치한 후에 옥상에서의 작업을 시작하도록 하여 안전을 최우선적으로 확보토록 한다.\n1.4.3 공사기간 중 선행공정으로 인한 공사 연기 사유가 발생할 경우, 선행공정의 지연기간 만큼 공사기간을 연장할 수 있다.\n1.4.4 공사가 긴급을 요하여 감독원이 필요하다고 인정하는 경우를 제외하고는 감독자의 승인없이 야간작업을 할 수 없다.', 'metadata': {'id': '4a61ae9d-43c7-42ce-8104-39bcf66c25be', 'source': 'LHCS 31 60 30 05 태양광 발전설비', 'question': '태양광 발전 설비 공사에서 옥상 안전난간대 설치 시, 건축공사분의 옥상 안전난간대와 겹칠 경우 어떻게 조치해야 하나요?', 'answer': '건축공사분의 옥상 안전난간대와 겹칠 경우, 건축공사분의 옥상 안전난간대를 설계변경 조치하고, 태양전지판이 설치되는 면만 안전난간대를 시공해야 합니다.'}}
{'page_content': '1.5 제출물\n1.5.1 일반사항\n(2) 제품자료는 골조공사 완료 전까지 제출, LH의 승인을 받은 후 제작하여야 한다.\n(3) 수급인은 아래 서류에 대하여 계약 체결일로부터 30일 이내에 제출일정 등에 대하여 감독자와 협의하고 제출하여야 한다.\n 공사착공 시 설치공정표, 동원인원계획표, 설치계획도, 감독원 선임계 및 안전관리 계획서 등을 감독자에게 제출하여야 한다.\n1.5.2 자재 제품자료\n(1) 제작도면\n 태양전지 모듈 어레이 제작도\n